<a href="https://colab.research.google.com/github/csw180/romanticpy/blob/main/datareader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas-datareader

In [ ]:
import pandas_datareader.data as web
import pandas as pd
import datetime

start = datetime.datetime(2021,10,1)
end = datetime.datetime(2022,2, 24)
# samsung_df = web.DataReader('005930', 'naver', start='2021-10-01', end='2022-02-24')
# samsung_df = web.DataReader('005930', 'naver',start='2021-01-01')
samsung_df = web.DataReader('230360', 'naver')
# samsung_df = web.DataReader('006890', 'naver')

samsung_df['ma5'] = samsung_df['Close'].rolling(window=5).mean()
samsung_df['ma20'] = samsung_df['Close'].rolling(window=20).mean()
samsung_df['vma20'] = samsung_df['Volume'].rolling(window=20).mean()
samsung_df['ma60'] = samsung_df['Close'].rolling(window=60).mean()
samsung_df['ma120'] = samsung_df['Close'].rolling(window=120).mean()
samsung_df['ma240'] = samsung_df['Close'].rolling(window=240).mean()

samsung_df['Open'] = samsung_df['Open'].astype(float, errors = 'raise')
samsung_df['High'] = samsung_df['High'].astype(float, errors = 'raise')
samsung_df['Low'] = samsung_df['Low'].astype(float, errors = 'raise')
samsung_df['Close'] = samsung_df['Close'].astype(float, errors = 'raise')
samsung_df['Volume'] = samsung_df['Volume'].astype(float, errors = 'raise')

# print(samsung_df)
samsung_df = samsung_df.dropna()
samsung_df.info()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9,5))
plt.plot(samsung_df.index, samsung_df['ma5'], label="MA5")
plt.plot(samsung_df.index, samsung_df['ma20'], label="MA20")
plt.plot(samsung_df.index, samsung_df['ma60'], label="MA60")
plt.plot(samsung_df.index, samsung_df['ma120'], label="MA120")
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
import numpy as np

samsung_df['asc'] = samsung_df['Close'] / samsung_df['Open'] - 1
samsung_df['ma20_asc']=samsung_df['ma20'] - samsung_df['ma20'].shift(1)
#20이평선 연속3봉 내림새이다가 금일 돌려세움. 거래량20이평보다 2배이상 터질것. 당일양봉일것
samsung_df['buy'] = np.where( (samsung_df['ma20_asc'] > 0) &\
          (samsung_df['ma20_asc'].shift(1) < 0) & (samsung_df['ma20_asc'].shift(2) < 0) & (samsung_df['ma20_asc'].shift(3) < 0) &\
          (samsung_df['Volume'] > samsung_df['vma20'] * 2.0) &\
          (samsung_df['asc'] > 0) , 'OK', '')
samsung_df = samsung_df[::-1]
samsung_df['maxhigh'] = samsung_df['High'].rolling(window=20).max()
samsung_df['status'] = np.where( (samsung_df['buy'] == 'OK') & ( (samsung_df['Close'] * 1.033) < samsung_df['maxhigh']),'Success','Fail' )
samsung_df = samsung_df[samsung_df['buy'] == 'OK']
# samsung_df['2021-11-05']

samsung_df